In [43]:
import pandas as pd

shoes = pd.read_csv('StockX-Data-Contest-2019.csv')

# get rid of any dollar sign and commas in price and convert to float to divide
shoes['Sale Price'] = shoes['Sale Price'].str.replace('[\$\,]|\.\d*', '').astype('float')
shoes['Retail Price'] = shoes['Retail Price'].str.replace('[\$\,]|\.\d*', '').astype('float')

# Create a "Price Ratio" column (SalePrice / RetailPrice = Price Ratio)
shoes['Price Ratio'] = shoes['Sale Price'] / shoes['Retail Price']

# Convert dates to pandas datetime
shoes['Order Date'] = pd.to_datetime(shoes['Order Date'])
shoes['Release Date'] = pd.to_datetime(shoes['Release Date'])

shoes.head(4)

/var/folders/y3/dlgpd0s57gx2x8t7lwlrczdm0000gn/T/ipykernel_42788/2051923040.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  shoes['Sale Price'] = shoes['Sale Price'].str.replace('[\$\,]|\.\d*', '').astype('float')
/var/folders/y3/dlgpd0s57gx2x8t7lwlrczdm0000gn/T/ipykernel_42788/2051923040.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  shoes['Retail Price'] = shoes['Retail Price'].str.replace('[\$\,]|\.\d*', '').astype('float')


,Order Date,Brand,Sneaker Name,Sale Price,Retail Price,Release Date,Shoe Size,Buyer Region,Price Ratio
0,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-Low-V2-Beluga,1097.0,220.0,2016-09-24,11.0,California,4.986364
1,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Copper,685.0,220.0,2016-11-23,11.0,California,3.113636
2,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Green,690.0,220.0,2016-11-23,11.0,California,3.136364
3,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Red,1075.0,220.0,2016-11-23,11.5,Kentucky,4.886364


Preparing data
TODO: Make sure all features are number based and between [0 - 1]

In [32]:
# Create training and validation splits
df_train = shoes.sample(frac=0.7, random_state=0)  # frac is the fraction of the data set to use
df_valid = shoes.drop(df_train.index)  # drops (slices) the training data from shoes using the index (shoes - training = valid)
print(df_train.head(4))

# Scale to [0, 1]
max_ = df_train.max(axis=0)
print(max_)
min_ = df_train.min(axis=0)
# df_train = (df_train - min_) / (max_ - min_)
# df_valid = (df_valid - min_) / (max_ - min_)

# Split features and target
X_train = df_train.drop('Sale Price', axis=1)
X_valid = df_valid.drop('Sale Price', axis=1)
y_train = df_train['Sale Price']
y_valid = df_valid['Sale Price']

print(shoes.head())
print(X_train.shape)

      Order Date   Brand                                  Sneaker Name  \
86246     1/6/19   Yeezy  Adidas-Yeezy-Boost-350-V2-Semi-Frozen-Yellow   
33080    6/30/18   Yeezy              adidas-Yeezy-Boost-350-V2-Butter   
39356    7/22/18   Yeezy              adidas-Yeezy-Boost-350-V2-Butter   
94586    1/31/19   Yeezy   adidas-Yeezy-Boost-350-V2-Static-Reflective   

       Sale Price  Retail Price Release Date  Shoe Size   Buyer Region  Profit  
86246         249           220     11/18/17        8.5       New York      29  
33080         290           220      6/30/18       12.0        Florida      70  
39356         288           220      6/30/18       14.0  West Virginia      68  
94586         488           220     12/26/18        4.0         Oregon     268  
Order Date                                           9/9/18
Brand                                             Off-White
Sneaker Name    adidas-Yeezy-Boost-350-V2-Static-Reflective
Sale Price                                  

A three-layer network with over 1500 neurons.
This network should be capable of learning fairly complex relationships in the data

In [33]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(units=512, activation='relu', input_shape=[7]),
    layers.Dense(units=512, activation='relu'),
    layers.Dense(units=512, activation='relu'),
    layers.Dense(1),
])

Compile in the optimizer and loss function.

In [34]:
# Adam is a great general-purpose optimizer. :)
# Adam is an SGD (Stochastic (Random) Gradient Descent) algo that has an adaptive learning rate (self-tuning) making it suitable for most problems.
model.compile(
    optimizer='adam',
    loss='mae',  # MAE (Mean Absolute Error) abs(y_true - y_pred)
)

Time to start training!

In [39]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=256, # Keras feeds the optimizer 256 rows of the training data at a time
    epochs=10,  # 10 times all the way through the dataset
)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).